## 02. 学習用データセットの前処理

このノートブックの目的：
1. `01_feature_engineering.ipynb`で作成したアイテムプロファイルを読み込む。
2. 数値特徴量をStandardScalerで標準化する。
3. 注文履歴から「同時に購入された商品ペア（ポジティブペア）」を生成する。
4. 上記2つを組み合わせて、sentence-transformersが学習に利用する`InputExample`オブジェクトを作成する。
5. 次の`03_model_training.ipynb`で利用する中間ファイルを保存する。

### 1. 準備：ライブラリとデータの読み込み

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import itertools
from sentence_transformers.readers import InputExample
import pickle
import os
from tqdm import tqdm

In [2]:
# --- 定数定義 ---
# 入力データが格納されているディレクトリ
DATA_DIR = './instacart_dataset/'
# 中間成果物を保存するディレクトリ
TMP_DATA_DIR = './tmp_data/'

# 入力ファイルパス
item_profiles_path = os.path.join(TMP_DATA_DIR, '01_item_profiles.csv')
order_products_path = os.path.join(DATA_DIR, 'order_products__prior.csv')

# 出力ファイルパス
scaled_profiles_path = os.path.join(TMP_DATA_DIR, '02_item_profiles_scaled.csv')
scaler_path = os.path.join(TMP_DATA_DIR, '02_scaler.pkl')
train_examples_path = os.path.join(TMP_DATA_DIR, '02_train_examples.pkl')

In [3]:
# --- データの読み込み ---
item_profiles_df = pd.read_csv(item_profiles_path)
order_products_df = pd.read_csv(order_products_path)

In [4]:
# --- 特徴量名の定義 ---
# 計画書で定義した特徴量名をリスト化
numerical_feature_names = [
    'reorder_rate', 'avg_add_to_cart_order', 'total_orders',
    'unique_users', 'avg_days_since_prior_order'
]
categorical_feature_names = ['aisle_id', 'department_id']

### 2. 数値特徴量の標準化

In [5]:
scaler = StandardScaler()

In [6]:
# 数値特徴量を抽出し、標準化を適用
item_profiles_df[numerical_feature_names] = scaler.fit_transform(
    item_profiles_df[numerical_feature_names]
)

### 3. 中間成果物の保存: 標準化済みプロファイルとScaler


In [7]:
# 標準化済みのアイテムプロファイルを保存
item_profiles_df.to_csv(scaled_profiles_path, index=False)

In [8]:
# 後で推論に使うために、学習済みScalerを保存
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)

### 4. InputExampleの作成

In [9]:
# --- パラメータ定義 ---
# 最終的に目標とするペアの総数
N_TARGET_PAIRS = 5_000_000
# 1バスケットから抽出するペア数の上限。ノイズの多い巨大バスケットの影響を抑制する
MAX_PAIRS_PER_BASKET = 10 
# 再現性のための乱数シード
RANDOM_STATE = 42

In [10]:
# order_idでグループ化し、各注文に含まれる商品のリストを作成
# tqdmを適用して進捗を表示
tqdm.pandas(desc="注文バスケットの作成")
baskets = order_products_df.groupby('order_id')['product_id'].progress_apply(list)

注文バスケットの作成: 100%|██████████| 3214874/3214874 [01:38<00:00, 32533.48it/s]


In [11]:
# 2アイテム未満のバスケットはペアを生成できないので除外
baskets = baskets[baskets.str.len() >= 2]
print(f"ペア生成対象のバスケット数: {len(baskets)}")

ペア生成対象のバスケット数: 3058126


In [12]:
# --- ペアの生成とサンプリング ---
# このアプローチの核心：
# 全ペアを一度に生成せず、バスケットごとにサンプリングしながらペアリストを構築する。
# これにより、巨大な中間データフレームの作成を回避し、メモリを節約する。

import random
random.seed(RANDOM_STATE) # randomモジュールのシードも固定

final_pairs = []
for basket in tqdm(baskets, desc="バスケットからペアをサンプリング中"):
    # バスケット内のアイテム数が多すぎても、生成するペアは上限を設ける
    if len(basket) > (MAX_PAIRS_PER_BASKET * 2): # 大まかなヒューリスティック
        # バスケットが巨大な場合、アイテム自体をサンプリングしてからペアを作る
        sampled_basket = random.sample(basket, k=20) 
    else:
        sampled_basket = basket
        
    # ペアを生成
    pairs = list(itertools.combinations(sampled_basket, 2))
    
    # 1バスケットから生成されたペア数が上限を超える場合は、さらにサンプリング
    if len(pairs) > MAX_PAIRS_PER_BASKET:
        pairs = random.sample(pairs, k=MAX_PAIRS_PER_BASKET)
        
    final_pairs.extend(pairs)

print(f"上限設定ありで生成されたペアの総数: {len(final_pairs)}")

バスケットからペアをサンプリング中: 100%|██████████| 3058126/3058126 [00:38<00:00, 79458.44it/s]

上限設定ありで生成されたペアの総数: 26560810


In [13]:
# --- 最終的なサンプリング ---
# 目標のペア数になるように全体から最終サンプリング
# これにより、特定のバスケットへの偏りをなくし、データセット全体から均一に抽出する
if len(final_pairs) > N_TARGET_PAIRS:
    print(f"最終目標の {N_TARGET_PAIRS:,} 件にランダムサンプリングします。")
    final_pairs = random.sample(final_pairs, k=N_TARGET_PAIRS)
else:
    print("生成されたペア数が目標より少ないため、全て使用します。")
    N_TARGET_PAIRS = len(final_pairs)

print(f"最終的な学習用ペアの数: {len(final_pairs)}")

最終目標の 5,000,000 件にランダムサンプリングします。
最終的な学習用ペアの数: 5000000


In [14]:
# --- InputExampleの作成 ---
# 高速な検索のためにproduct_idをインデックスに設定
item_profiles_df.set_index('product_id', inplace=True)

train_examples = []
for item_a_id, item_b_id in tqdm(final_pairs, desc="InputExampleの作成"):
    try:
        item_a_features = item_profiles_df.loc[item_a_id].to_dict()
        item_b_features = item_profiles_df.loc[item_b_id].to_dict()
        example = InputExample(texts=[item_a_features, item_b_features])
        train_examples.append(example)
    except KeyError:
        continue
        
print(f"作成された学習サンプルの数: {len(train_examples)}")

InputExampleの作成: 100%|██████████| 5000000/5000000 [20:49<00:00, 4001.98it/s] 

作成された学習サンプルの数: 5000000


### 5. 中間成果物の保存: 学習用サンプル

In [15]:
with open(train_examples_path, 'wb') as f:
    pickle.dump(train_examples, f)

print(f"学習用InputExampleリストを '{train_examples_path}' に保存しました。")

学習用InputExampleリストを './tmp_data/02_train_examples.pkl' に保存しました。
